In [2]:
from qmlab.preprocessing import parse_biomed_data_to_ndarray, reduce_feature_dim
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC

random_state = 12345

X, y, feature_names = parse_biomed_data_to_ndarray("sobar_new")
X = X[:, :4]
skf = StratifiedKFold(n_splits=5, random_state=random_state, shuffle=True)
clf = SVC(kernel="linear", random_state=random_state)

scores = []
for train_idx, test_idx in skf.split(X, y):
    X_train = X[train_idx, :]
    X_test = X[test_idx,: ]
    y_train = y[train_idx]
    y_test = y[test_idx]

    clf.fit(X_train, y_train)

    acc_score = clf.score(X_test, y_test)
    scores.append(acc_score)

avg_acc_score = sum(scores) / len(scores)
print(avg_acc_score)

0.8057142857142857


In [3]:
from qiskit.circuit.library import ZZFeatureMap
from sklearn.model_selection import ShuffleSplit
from qmlab.preprocessing import scale_to_specified_range
from qiskit_machine_learning.kernels import FidelityQuantumKernel

X, y, _ = parse_biomed_data_to_ndarray("sobar_new")
X = X[:, :2]
X = scale_to_specified_range(X, range=(-1.0, 1.0))
qfm = ZZFeatureMap(feature_dimension=X.shape[1], reps=2)
quantum_kernel = FidelityQuantumKernel(feature_map=qfm)
qsvm = SVC(kernel="precomputed", random_state=random_state)
rs = ShuffleSplit(n_splits=10, test_size=0.3, random_state=random_state)

scores = []
for train_idx, test_idx in rs.split(X):
    X_train = X[train_idx, :]
    y_train = y[train_idx]
    X_test = X[test_idx, :]
    y_test = y[test_idx]

    kernel_train = quantum_kernel.evaluate(X_train)
    kernel_test = quantum_kernel.evaluate(X_test, X_train)
    qsvm.fit(kernel_train, y_train)
    score = qsvm.score(kernel_test, y_test)

    scores.append(score)

avg_score = sum(scores) / len(scores)

In [4]:
print(avg_score)

0.7
